## Import libraries

In [1]:
import os

files = os.listdir('.')
print(files)

['shopping_list.json', 'shopping_guide.md', 'Social_Media_Strategist_Agent.py', 'leftover.py', '.env', '__pycache__', 'CrewAI_1.ipynb', '.ipynb_checkpoints', 'Course', 'requirements.txt', 'config', 'MealGroceryPlanner.py', 'Meal & Grocery Planner.ipynb', 'meals.json']


In [2]:
from pydantic import BaseModel, Field
from typing import List, Dict, Optional, Literal
from IPython.display import display, JSON, Markdown
from datetime import datetime
from leftover import LeftoversCrew
import os

## Our Grocery Shopping Data Structure

1. ```GroceryItem```         → Individual grocery item with details
2. ```MealPlan```           → Recipe information with researched ingredients  
3. ```ShoppingCategory```   → Store section with organized items
4. ```GroceryShoppingPlan``` → Complete shopping strategy with budget analysis

### 1. GroceryItem

In [3]:
class GroceryItem(BaseModel):
    """Individual grocery item"""
    name: str = Field(description="Name of the grocery item")
    quantity: str = Field(description="Quantity needed (for example, '2 kg', '200 gram')")
    estimated_price: str = Field(description="Estimated price (for example, '₹100-500')")
    category: str = Field(description="Store section (for example, 'Produce', 'Dairy')")

In [4]:
sample_item = GroceryItem(
    name="Kheer",
    quantity="250 gram",
    estimated_price="₹100",
    category="Dairy"
)

sample_item

GroceryItem(name='Kheer', quantity='250 gram', estimated_price='₹100', category='Dairy')

In [5]:
type(sample_item)

__main__.GroceryItem

In [6]:
# Display structured data
print("🛒 Sample Grocery Item Structure:")
display(JSON(sample_item.model_dump()))

🛒 Sample Grocery Item Structure:


<IPython.core.display.JSON object>

### 2. MealPlan

In [7]:
class MealPlan(BaseModel):
    """Simple meal plan"""
    meal_name: str = Field(description="Name of the meal")
    taste: str = Field(description="Taste profile of the meal 'Salty', 'Sweet', 'Sour'")
    servings: int = Field(description="Number of people it serves")
    researched_ingredients: List[str] = Field(description="Ingredients found through research")


In [8]:
# Corrected sample
sample_meal = MealPlan(
    meal_name="Samosa",
    taste="Salty",  # Now valid with Literal constraint
    servings=2,
    researched_ingredients=["Potato", "Paneer", "Peas", "Spices"]  # Completed the list example
)

In [9]:
print("\n🍽️ Sample Meal Plan Structure:")
display(JSON(sample_meal.model_dump()))


🍽️ Sample Meal Plan Structure:


<IPython.core.display.JSON object>

### 3. ShoppingCategory

In [10]:
class ShoppingCategory(BaseModel):
    """Store section with items"""
    section_name: str = Field(description="Store section (for example, 'Produce', 'Dairy')")
    items: List[GroceryItem] = Field(description="Items in this section")
    estimated_total: str = Field(description="Estimated cost for this section in Indian Rupees (₹)")

In [11]:
produce_section = ShoppingCategory(
    section_name="Produce",
    items=[
        GroceryItem(
            name="Potatoes",
            quantity="2 kg",
            estimated_price="₹91-100",  # Range format as per description
            category="Produce"
        ),
        GroceryItem(
            name="Tomatoes",
            quantity="1 kg",
            estimated_price="₹55-65",
            category="Produce"
        )
    ],
    estimated_total="₹150-170"  # Approximate sum range for section
)

dairy_section = ShoppingCategory(
    section_name="Dairy",
    items=[sample_item],
    estimated_total="₹100"
)


In [12]:
print("\n🏪 Sample Shopping Section:")
display(JSON(produce_section.model_dump()))


🏪 Sample Shopping Section:


<IPython.core.display.JSON object>

### 4. GroceryShoppingPlan

In [13]:
class GroceryShoppingPlan(BaseModel):
    """Complete simplified shopping plan"""
    total_budget: str = Field(description="Total planned budget")
    meal_plans: List[MealPlan] = Field(description="Planned meals")
    shopping_sections: List[ShoppingCategory] = Field(description="Organized by store sections")
    shopping_tips: List[str] = Field(description="Money-saving and efficiency tips")

In [14]:
weekly_shopping_plan = GroceryShoppingPlan(
    total_budget="₹50-200",
    meal_plans=[sample_meal],
    shopping_sections=[produce_section, dairy_section],  # Define dairy_section first
    shopping_tips=["Shop sales in NÄrnaund markets.", "Prefer local dairy for freshness.", "Batch buy staples."]
)

weekly_shopping_plan

GroceryShoppingPlan(total_budget='₹50-200', meal_plans=[MealPlan(meal_name='Samosa', taste='Salty', servings=2, researched_ingredients=['Potato', 'Paneer', 'Peas', 'Spices'])], shopping_sections=[ShoppingCategory(section_name='Produce', items=[GroceryItem(name='Potatoes', quantity='2 kg', estimated_price='₹91-100', category='Produce'), GroceryItem(name='Tomatoes', quantity='1 kg', estimated_price='₹55-65', category='Produce')], estimated_total='₹150-170'), ShoppingCategory(section_name='Dairy', items=[GroceryItem(name='Kheer', quantity='250 gram', estimated_price='₹100', category='Dairy')], estimated_total='₹100')], shopping_tips=['Shop sales in NÄ\x81rnaund markets.', 'Prefer local dairy for freshness.', 'Batch buy staples.'])

## Setting Up Our LLM and Essential Tools  

In [15]:
import os
from crewai_tools import SerperDevTool
from crewai import Agent, Task, Crew, Process
from crewai import LLM
from dotenv import load_dotenv

# Set environment variables
load_dotenv()

True

In [16]:
perplexity_llm = LLM(
    model="sonar-pro", 
    base_url="https://api.perplexity.ai",
    api_key=os.getenv("PERPLEXITY_API_KEY"),  # Required!
    temperature=0.3,
    max_tokens=2500
)

# Local Ollama LLM fallback (if needed)
ollama_llm = LLM(
    model="ollama/llama3.2:latest",
    base_url="http://localhost:11434",
    temperature=0.3,
    max_tokens=100
)

llm = perplexity_llm

## 1. Meal Planner

In [17]:
meal_planner = Agent(
    role="Meal Planner & Recipe Researcher",
    goal="Search for optimal recipes and create detailed meal plans",
    backstory="A skilled meal planner who researches the best recipes online, considering dietary needs, taste preferences, and budget constraints.",
    tools=[SerperDevTool()],
    llm=llm,
    verbose=True)

In [18]:
meal_planning_task = Task(
    description=(
        "Search for the best '{meal_name}' recipe for {servings} people within a {budget} budget. "
        "Consider dietary restrictions: {dietary_restrictions} and Taste preference : {taste}. "
        "Find recipes that match the Taste preference and provide complete ingredient lists with quantities."
    ),
    expected_output="A detailed meal plan with researched ingredients, quantities, and cooking instructions appropriate for the skill level.",
    agent=meal_planner,
    output_pydantic=MealPlan,
    output_file="meals.json"
)

### Creating and Running Our Meal Planning Crew

In [19]:
from crewai import Crew, Process

meal_planner_crew = Crew(
    agents=[meal_planner],
    tasks=[meal_planning_task],
    process=Process.sequential,  # Ensures tasks are executed in order
    verbose=True
)

meal_planner_result = meal_planner_crew.kickoff(
    inputs={
        "meal_name": "Samosa",
        "servings": 2,
        "budget": "₹25",                           
        "dietary_restrictions": ["chili"],       
        "taste": "Sweet"                
    }
)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 21e31096-3f94-448d-a48a-5f674ab4815d                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│                                                                                                                 │
│  Task: Search for the best 'Samosa' recipe for 2 people within a ₹25 budget. Consider dietary restrictions:     │
│  ['chili'] and Taste preference : Sweet. Find recipes that match the Taste preference and provide complete      │
│  ingredient lists with quantities.                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "meal_name": "Mini Sweet Samosas with Cream Cheese and Rose Sugar Syrup",                                    │
│    "taste": "Sweet",                                                                                            │
│    "servings": 2,                                                                                               │
│    "researched_ingredients": [                                                                                  │
│      "Frozen samosa sheets - 1 package (yields approximately 8-12 mini samosas)",                               │
│      "Cream cheese - 100 grams (6 cubes or equivalent)",                                                        │
│      "Sugar - ½ cup (100 grams)",                                                                               │
│      "Water - ¼ cup (60 ml)",                                                                                   │
│      "Rose water - 1 tablespoon",                                                                               │
│      "Pistachio - 2 tablespoons (powdered or chopped)",                                                         │
│      "Oil for deep frying - as needed",                                                                         │
│      "All-purpose flour - 1 tablespoon (for sealing paste)",                                                    │
│      "Lemon juice - a squeeze (optional, for sugar syrup)"                                                      │
│    ],                                                                                                           │
│    "cooking_instructions": [                                                                                    │
│      "Prepare the sugar syrup: In a pan, add ½ cup sugar, ¼ cup water, and a squeeze of lemon juice. Bring to   │
│  a boil, stirring if needed. Once boiling, simmer for 5-6 minutes until slightly thickened. Turn off flame and  │
│  add 1 tablespoon rose water. Set aside to cool.",                                                              │
│      "Prepare the filling: If using cream cheese cubes, cut each cube into 6 small squares. You will need       │
│  approximately 1 small piece per samosa.",                                                                      │
│      "Assemble the samosas: Take out frozen samosa sheets and keep them in a stack. Each sheet yields 4 mini    │
│  samosas. Place a small piece of cream cheese in the center of each samosa sheet.",                             │
│      "Seal the samosas: Mix 1 tablespoon all-purpose flour with a little water to create a paste. Use this to   │
│  seal the edges of each samosa.",                                                                               │
│      "Fry the samosas: Heat oil in a deep pan or wok on medium heat. Fry samosas a few at a time, turning       │
│  occasionally, until golden brown and crispy. Avoid high heat to prevent the filling from oozing out.",         │
│      "Drain and coat: Remove fried samosas with a slotted spoon and drain on paper towels. While still warm,    │
│  dip each samosa into the prepared rose sugar syrup.",                                                          │
│      "Garnish and serve: Arrange on a plate and sprinkl

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: ff283d0a-c9a2-4d10-85a8-9f482558ad6a                                                                     │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────── Trace Batch Finalization ────────────────────────────────────────────╮
│ ✅ Trace batch finalized with session ID: f9a56d8e-906f-42d8-9e87-5f0494d314d8                                  │
│                                                                                                                 │
│ 🔗 View here:                                                                                                   │
│ https://app.crewai.com/crewai_plus/ephemeral_trace_batches/f9a56d8e-906f-42d8-9e87-5f0494d314d8?access_code=TRA │
│ CE-d140f5b0c5                                                                                                   │
│ 🔑 Access Code: TRACE-d140f5b0c5                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 21e31096-3f94-448d-a48a-5f674ab4815d                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│    "meal_name": "Mini Sweet Samosas with Cream Cheese and Rose Sugar Syrup",                                    │
│    "taste": "Sweet",                                                                                            │
│    "servings": 2,                                                                                               │
│    "researched_ingredients": [                                                                                  │
│      "Frozen samosa sheets - 1 package (yields approximately 8-12 mini samosas)",                               │
│      "Cream cheese - 100 grams (6 cubes or equivalent)",                                                        │
│      "Sugar - ½ cup (100 grams)",                                                                               │
│      "Water - ¼ cup (60 ml)",                                                                                   │
│      "Rose water - 1 tablespoon",                                                                               │
│      "Pistachio - 2 tablespoons (powdered or chopped)",                                                         │
│      "Oil for deep frying - as needed",                                                                         │
│      "All-purpose flour - 1 tablespoon (for sealing paste)",                                                    │
│      "Lemon juice - a squeeze (optional, for sugar syrup)"                                                      │
│    ],                                                                                                           │
│    "cooking_instructions": [                                                                                    │
│      "Prepare the sugar syrup: In a pan, add ½ cup sugar, ¼ cup water, and a squeeze of lemon juice. Bring to   │
│  a boil, stirring if needed. Once boiling, simmer for 5-6 minutes until slightly thickened. Turn off flame and  │
│  add 1 tablespoon rose water. Set aside to cool.",                                                              │
│      "Prepare the filling: If using cream cheese cubes, cut each cube into 6 small squares. You will need       │
│  approximately 1 small piece per samosa.",                                                                      │
│      "Assemble the samosas: Take out frozen samosa sheets and keep them in a stack. Each sheet yields 4 mini    │
│  samosas. Place a small piece of cream cheese in the center of each samosa sheet.",                             │
│      "Seal the samosas: Mix 1 tablespoon all-purpose flour with a little water to create a paste. Use this to   │
│  seal the edges of each samosa.",                                                                               │
│      "Fry the samosas: Heat oil in a deep pan or wok on medium heat. Fry samosas a few at a time, turning       │
│  occasionally, until golden brown and crispy. Avoid high heat to prevent the filling from oozing out.",         │
│      "Drain and coat: Remove fried samosas with a slotted spoon and drain on paper towels. While still warm,    │
│  dip each samosa into the prepared rose sugar syrup.",

In [20]:
print("✅ Single meal planning completed!")
print("📋 Single Meal Results:")
print(meal_planner_result)

✅ Single meal planning completed!
📋 Single Meal Results:
meal_name='Mini Sweet Samosas with Cream Cheese and Rose Sugar Syrup' taste='Sweet' servings=2 researched_ingredients=['Frozen samosa sheets - 1 package (yields approximately 8-12 mini samosas)', 'Cream cheese - 100 grams (6 cubes or equivalent)', 'Sugar - ½ cup (100 grams)', 'Water - ¼ cup (60 ml)', 'Rose water - 1 tablespoon', 'Pistachio - 2 tablespoons (powdered or chopped)', 'Oil for deep frying - as needed', 'All-purpose flour - 1 tablespoon (for sealing paste)', 'Lemon juice - a squeeze (optional, for sugar syrup)']


## 2. Shopping Organiser

In [21]:
shopping_organizer = Agent(
    role="Shopping Organizer", 
    goal="Organize grocery lists by store sections efficiently",
    backstory="An experienced shopper who knows how to organize lists for quick store trips and considers dietary restrictions.",
    tools=[],
    llm=llm,
    verbose=False
)

In [22]:
shopping_task = Task(
    description=(
        "Organize the ingredients from the '{meal_name}' meal plan into a grocery shopping list. "
        "Group items by store sections and estimate quantities for {servings} people. "
        "Consider dietary restrictions: {dietary_restrictions} and Taste Preferences: {taste}. "
        "Stay within budget: {budget}."
    ),
    expected_output="An organized shopping list grouped by store sections with quantities and prices.",
    agent=shopping_organizer,
    context=[meal_planning_task],
    output_pydantic=GroceryShoppingPlan,
    output_file="shopping_list.json"
)

### Building Our Two-Agent Grocery Crew

Now we're ready to create our first multi-agent workflow. This crew combines both the meal planner and shopping organizer to create a complete meal-to-shopping-list pipeline.

**How Multi-Agent Workflows Work:**
1. **Sequential Processing**: The `Process.sequential` ensures agents work in order - meal planning first, then shopping organization
2. **Data Flow**: The shopping organizer automatically receives the meal planner's output through the task context
3. **Structured Output**: The final result follows our `GroceryShoppingPlan` Pydantic model for consistent formatting

**Benefits of This Approach:**
- **Separation of Concerns**: Each agent has a specific role and expertise
- **Reusability**: We can swap out agents or add new ones without changing the overall workflow
- **Reliability**: Pydantic ensures our output is always properly structured


In [23]:
two_agent_grocery_crew = Crew(
    agents=[meal_planner, shopping_organizer],  # Both agents
    tasks=[meal_planning_task, shopping_task],   # Both tasks
    process=Process.sequential,
    verbose=True,
    memory = False,
)

# Run the complete crew (this will do BOTH meal planning AND shopping)
shopping_result = two_agent_grocery_crew.kickoff(
    inputs={
        "meal_name": "Samosa Chaat",
        "servings": 4,
        "budget": "₹200",                           
        "dietary_restrictions": ["no chilli"],      
        "taste": "salty"               
    }
)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 502852ef-c3b0-413f-8744-635d69a97657                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│                                                                                                                 │
│  Task: Search for the best 'Samosa Chaat' recipe for 4 people within a ₹200 budget. Consider dietary            │
│  restrictions: ['no chilli'] and Taste preference : salty. Find recipes that match the Taste preference and     │
│  provide complete ingredient lists with quantities.                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│                                                                                                                 │
│  Thought: Thought: The search results provide several Samosa Chaat recipes, but all include chillies or chilli  │
│  powder, violating the 'no chilli' restriction. I need to search for recipes specifically without chilli to     │
│  match the criteria, scaled for 4 people, salty taste, under ₹200 budget. I'll craft a search query for         │
│  no-chilli versions and check costs implicitly through common ingredients.                                      │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  {                                                                                                              │
│    "search_query": "samosa chaat recipe no chilli no chili no spice mild for 4 people salty indian street       │
│  food"                                                                                                          │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Shopping Organizer                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {"total_budget":"₹180","meal_plans":[{"meal_name":"Samosa Chaat (No Chilli                                     │
│  Version)","taste":"Salty","servings":4,"researched_ingredients":["8 medium-sized samosas (store-bought or      │
│  homemade)","1 can (19 oz) chickpeas, drained and rinsed","2.5 cups water","1 tablespoon cooking oil or         │
│  ghee","1 teaspoon cumin seeds","½ teaspoon asafoetida (hing)","1 tablespoon ginger paste","3-4 cloves garlic,  │
│  minced","1 tablespoon chopped cilantro stems","1 teaspoon coriander powder","¾ teaspoon cumin powder","1       │
│  teaspoon garam masala","1 teaspoon black salt (kala namak)","Salt to taste","1 tablespoon kasoori methi        │
│  (dried fenugreek leaves)","1 cup plain yogurt, whisked","½ cup tamarind pulp (strained) for tamarind           │
│  chutney","3 tablespoons sugar for tamarind chutney","1 teaspoon roasted cumin powder (bhunna zeera)","1        │
│  teaspoon roasted coriander powder (bhunna dhania)","½ teaspoon Himalayan black salt for chutney","3            │
│  tablespoons ketchup for tamarind chutney","1 cup fresh cilantro, chopped","1 cup sev (crispy chickpea          │
│  noodles)","1 medium onion, finely sliced","1 tablespoon lemon juice","1 teaspoon chaat                         │
│  masala"]}],"shopping_sections":[{"section_name":"Produce","items":[{"name":"Fresh cilantro","quantity":"1      │
│  bunch (50g)","estimated_price":"₹10","category":"Produce"},{"name":"Onion","quantity":"1                       │
│  medium","estimated_price":"₹10","category":"Produce"},{"name":"Lemon","quantity":"1","estimated_price":"₹10",  │
│  "category":"Produce"}],"estimated_total":"₹30"},{"section_name":"Dairy","items":[{"name":"Plain                │
│  yogurt","quantity":"500g","estimated_price":"₹40","category":"Dairy"}],"estimated_total":"₹40"},{"section_nam  │
│  e":"Canned & Packaged","items":[{"name":"Chickpeas (canned)","quantity":"1 can                                 │
│  (400g)","estimated_price":"₹50","category":"Canned & Packaged"},{"name":"Tamarind pulp","quantity":"100g       │
│  pack","estimated_price":"₹20","category":"Canned &                                                             │
│  Packaged"}],"estimated_total":"₹70"},{"section_name":"Bakery & Snacks","items":[{"name":"Samosas               │
│  (store-bought)","quantity":"8 medium","estimated_price":"₹30","category":"Bakery & Snacks"},{"name":"Sev       │
│  (crispy chickpea noodles)","quantity":"100g","estimated_price":"₹15","category":"Bakery &                      │
│  Snacks"}],"estimated_total":"₹45"},{"section_name":"Spices & Condiments","items":[{"name":"Cooking oil or      │
│  ghee","quantity":"50ml","estimated_price":"₹10","category":"Spices & Condiments"},{"name":"Cumin               │
│  seeds","quantity":"10g","estimated_price":"₹5","category":"Spices & Condiments"},{"name":"Asafoetida           │
│  (hing)","quantity":"5g","estimated_price":"₹5","category":"Spices & Condiments"},{"name":"Ginger               │
│  paste","quantity":"20g","estimated_price":"₹5","category":"Spices &                                            │
│  Condiments"},{"name":"Garlic","quantity":"1 small bulb","estimated_price":"₹5","category":"Spices &            │
│  Condiments"},{"name":"Coriander powder","quantity":"20g","estimated_price":"₹5","category":"Spices &           │
│  Condiments"},{"name":"Cumin powder","quantity":"20g","

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 702a0370-c659-47b9-a8a4-3f4abaa7441a                                                                     │
│  Agent: Shopping Organizer                                                                                      │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 502852ef-c3b0-413f-8744-635d69a97657                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {"total_budget":"₹180","meal_plans":[{"meal_name":"Samosa Chaat (No Chilli                       │
│  Version)","taste":"Salty","servings":4,"researched_ingredients":["8 medium-sized samosas (store-bought or      │
│  homemade)","1 can (19 oz) chickpeas, drained and rinsed","2.5 cups water","1 tablespoon cooking oil or         │
│  ghee","1 teaspoon cumin seeds","½ teaspoon asafoetida (hing)","1 tablespoon ginger paste","3-4 cloves garlic,  │
│  minced","1 tablespoon chopped cilantro stems","1 teaspoon coriander powder","¾ teaspoon cumin powder","1       │
│  teaspoon garam masala","1 teaspoon black salt (kala namak)","Salt to taste","1 tablespoon kasoori methi        │
│  (dried fenugreek leaves)","1 cup plain yogurt, whisked","½ cup tamarind pulp (strained) for tamarind           │
│  chutney","3 tablespoons sugar for tamarind chutney","1 teaspoon roasted cumin powder (bhunna zeera)","1        │
│  teaspoon roasted coriander powder (bhunna dhania)","½ teaspoon Himalayan black salt for chutney","3            │
│  tablespoons ketchup for tamarind chutney","1 cup fresh cilantro, chopped","1 cup sev (crispy chickpea          │
│  noodles)","1 medium onion, finely sliced","1 tablespoon lemon juice","1 teaspoon chaat                         │
│  masala"]}],"shopping_sections":[{"section_name":"Produce","items":[{"name":"Fresh cilantro","quantity":"1      │
│  bunch (50g)","estimated_price":"₹10","category":"Produce"},{"name":"Onion","quantity":"1                       │
│  medium","estimated_price":"₹10","category":"Produce"},{"name":"Lemon","quantity":"1","estimated_price":"₹10",  │
│  "category":"Produce"}],"estimated_total":"₹30"},{"section_name":"Dairy","items":[{"name":"Plain                │
│  yogurt","quantity":"500g","estimated_price":"₹40","category":"Dairy"}],"estimated_total":"₹40"},{"section_nam  │
│  e":"Canned & Packaged","items":[{"name":"Chickpeas (canned)","quantity":"1 can                                 │
│  (400g)","estimated_price":"₹50","category":"Canned & Packaged"},{"name":"Tamarind pulp","quantity":"100g       │
│  pack","estimated_price":"₹20","category":"Canned &                                                             │
│  Packaged"}],"estimated_total":"₹70"},{"section_name":"Bakery & Snacks","items":[{"name":"Samosas               │
│  (store-bought)","quantity":"8 medium","estimated_price":"₹30","category":"Bakery & Snacks"},{"name":"Sev       │
│  (crispy chickpea noodles)","quantity":"100g","estimated_price":"₹15","category":"Bakery &                      │
│  Snacks"}],"estimated_total":"₹45"},{"section_name":"Spices & Condiments","items":[{"name":"Cooking oil or      │
│  ghee","quantity":"50ml","estimated_price":"₹10","category":"Spices & Condiments"},{"name":"Cumin               │
│  seeds","quantity":"10g","estimated_price":"₹5","category":"Spices & Condiments"},{"name":"Asafoetida           │
│  (hing)","quantity":"5g","estimated_price":"₹5","category":"Spices & Condiments"},{"name":"Ginger               │
│  paste","quantity":"20g","estimated_price":"₹5","category":"Spices &                                            │
│  Condiments"},{"name":"Garlic","quantity":"1 small bulb","estimated_price":"₹5","category":"Spices &            │
│  Condiments"},{"name":"Coriander powder","quantity":"2

In [24]:
# Print the shopping results
print("✅ Complete meal planning + shopping completed!")
print("📋 Shopping Results:")
print(shopping_result)

✅ Complete meal planning + shopping completed!
📋 Shopping Results:
total_budget='₹180' meal_plans=[MealPlan(meal_name='Samosa Chaat (No Chilli Version)', taste='Salty', servings=4, researched_ingredients=['8 medium-sized samosas (store-bought or homemade)', '1 can (19 oz) chickpeas, drained and rinsed', '2.5 cups water', '1 tablespoon cooking oil or ghee', '1 teaspoon cumin seeds', '½ teaspoon asafoetida (hing)', '1 tablespoon ginger paste', '3-4 cloves garlic, minced', '1 tablespoon chopped cilantro stems', '1 teaspoon coriander powder', '¾ teaspoon cumin powder', '1 teaspoon garam masala', '1 teaspoon black salt (kala namak)', 'Salt to taste', '1 tablespoon kasoori methi (dried fenugreek leaves)', '1 cup plain yogurt, whisked', '½ cup tamarind pulp (strained) for tamarind chutney', '3 tablespoons sugar for tamarind chutney', '1 teaspoon roasted cumin powder (bhunna zeera)', '1 teaspoon roasted coriander powder (bhunna dhania)', '½ teaspoon Himalayan black salt for chutney', '3 tables

## 3. Budget Advisor

In [25]:
budget_advisor = Agent(
    role="Budget Advisor",
    goal="Provide cost estimates and money-saving tips",
    backstory="A budget-conscious shopper who helps families save money on groceries while respecting dietary needs.",
    tools=[SerperDevTool()],
    llm=llm,
    verbose=False
)

### Defining the Budget Analysis Task

In [26]:
budget_task = Task(
    description=(
        "Analyze the shopping plan for '{meal_name}' serving {servings} people. "
        "Ensure total cost stays within {budget}. Consider dietary restrictions: {dietary_restrictions}. "
        "Provide practical money-saving tips and alternative ingredients if needed to meet budget."
    ),
    expected_output="A complete shopping guide with detailed prices, budget analysis, and money-saving tips.",
    agent=budget_advisor,
    context=[meal_planning_task, shopping_task],
    output_file="shopping_guide.md"
)

In [27]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/-5NXnHGkX1WG81tqE8kSTg/agents.yaml"

--2026-01-04 16:55:29--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/-5NXnHGkX1WG81tqE8kSTg/agents.yaml
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 337 [text/x-yaml]
Saving to: ‘agents.yaml’

agents.yaml         100%[===================>]     337  --.-KB/s    in 0s      

2026-01-04 16:55:32 (275 MB/s) - ‘agents.yaml’ saved [337/337]



In [28]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/7fhq1KPxi7cuqjOCBgcmXQ/tasks.yaml"

--2026-01-04 16:55:32--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/7fhq1KPxi7cuqjOCBgcmXQ/tasks.yaml
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 517 [text/x-yaml]
Saving to: ‘tasks.yaml’

tasks.yaml          100%[===================>]     517  --.-KB/s    in 0s      

2026-01-04 16:55:33 (136 MB/s) - ‘tasks.yaml’ saved [517/517]



In [29]:
%mkdir -p config
%mv agents.yaml  config/agents.yaml
%mv tasks.yaml   config/tasks.yaml

### Using CrewBase and Decorators with CrewAI

#### What is `@CrewBase`?
In CrewAI, `@CrewBase` is a Python class decorator that automates the collection and wiring of your agents and tasks — especially when you're organizing your crew logic in Python, YAML or hybrid config (YAML + code). Basically, when we have the following:

```python
from crewai.project import CrewBase
from crewai.project.annotations import agent, task

@CrewBase
class myCrew:
    ...
```

#### Decorators overview
- CrewAI provides several decorators that are used to mark methods within your crew class for special handling.
| Decorator           | Marks…                                            |
| :------------------ | :----------------------------------------------- |
| `@CrewBase`         | The class that holds your agents & tasks         |
| `@agent`            | A method that returns an `Agent` object          |
| `@task`             | A method that returns a `Task` object            |
| `@crew`             | (Optional) A method that returns a `Crew` object |
| `@before_kickoff`   | (Optional) Runs once **before** the crew starts  |
| `@after_kickoff`    | (Optional) Runs once **after** the crew finishes |




In [30]:
from leftover import LeftoversCrew

leftovers_cb = LeftoversCrew(llm=llm)
yaml_leftover_manager = leftovers_cb.leftover_manager()
yaml_leftover_task    = leftovers_cb.leftover_task()

In [31]:
summary_agent = Agent(
    role="Report Compiler",
    goal="Compile comprehensive meal planning reports from all team outputs",
    backstory="A skilled coordinator who organizes information from multiple specialists into comprehensive, easy-to-follow reports.",
    tools=[],
    llm=perplexity_llm,
    verbose=False
)

In [32]:
summary_task = Task(
    description=(
        "Compile a comprehensive meal planning report that includes:\n"
        "1. The complete recipe and cooking instructions from the meal planner\n"
        "2. The organized shopping list with prices from the shopping organizer\n"
        "3. The budget analysis and money-saving tips from the budget advisor\n"
        "4. The leftover management suggestions from the waste reduction specialist\n"
        "Format this as a complete, user-friendly meal planning guide."
    ),
    expected_output="A comprehensive meal planning guide that combines all team outputs into one cohesive report.",
    agent=summary_agent,
    context=[meal_planning_task, shopping_task, budget_task, yaml_leftover_task],
)

In [33]:
complete_grocery_crew = Crew(
    agents=[
        meal_planner,           
        shopping_organizer,      
        budget_advisor,         
        yaml_leftover_manager,  # YAML-based leftover manager
        summary_agent           # New summary agent
    ],
    tasks=[
        meal_planning_task,     
        shopping_task,          
        budget_task,            
        yaml_leftover_task,    # YAML-based leftover task
        summary_task            # New summary task
    ],
    process=Process.sequential,
    verbose=True
)

In [34]:
# Run the complete crew
complete_result = complete_grocery_crew.kickoff(
    inputs={
        "meal_name": "Samosa Chaat",
        "servings": 2,
        "budget": "₹200",
        "dietary_restrictions": ["no chilli","No Mirch"],
        "taste": "Salty"
    }
)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 8f949483-1d91-4fbe-9c48-7110280105e0                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│                                                                                                                 │
│  Task: Search for the best 'Samosa Chaat' recipe for 2 people within a ₹200 budget. Consider dietary            │
│  restrictions: ['no chilli', 'No Mirch'] and Taste preference : Salty. Find recipes that match the Taste        │
│  preference and provide complete ingredient lists with quantities.                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│                                                                                                                 │
│  Thought: Thought: The search results provide several Samosa Chaat recipes, but all include chili, mirch, or    │
│  red chili powder, violating the 'no chilli, No Mirch' restriction. I need to search specifically for no-chili  │
│  versions adjusted for salty taste, scaled for 2 people under ₹200 budget, with complete ingredients and        │
│  instructions.                                                                                                  │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: leftover_task                                                                                            │
│  Agent: Leftover & Waste Reduction Specialist                                                                   │
│                                                                                                                 │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Report Compiler                                                                                         │
│                                                                                                                 │
│  Task: Compile a comprehensive meal planning report that includes:                                              │
│  1. The complete recipe and cooking instructions from the meal planner                                          │
│  2. The organized shopping list with prices from the shopping organizer                                         │
│  3. The budget analysis and money-saving tips from the budget advisor                                           │
│  4. The leftover management suggestions from the waste reduction specialist                                     │
│  Format this as a complete, user-friendly meal planning guide.                                                  │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

Output()

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 14cfc46f-7ade-4db0-801e-825a48dbaa8d                                                                     │
│  Agent: Report Compiler                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [35]:
print("✅ Complete meal planning with summary completed!")
print("📋 Complete Results:")
print(complete_result)

✅ Complete meal planning with summary completed!
📋 Complete Results:
# Comprehensive Meal Planning Guide: Samosa Chaat (No Chilli Version)

**Meal Overview**  
**Samosa Chaat (No Chilli Version)** is a flavorful, **salty** street-food inspired dish for **2 servings**, featuring warm samosas topped with spiced chickpeas (chole), yogurt, mild chutneys, crunchy sev, and fresh garnishes. This **chilli-free** recipe uses ketchup-based red chutney and no-chilli cilantro-mint chutney for mild tang. Total prep/cook time: ~20-30 minutes. **Total Budget: ₹200** (fits exactly with small packs and pantry staples). Enjoy guilt-free home-style chaat!

## 1. Complete Recipe & Cooking Instructions

### Ingredients (for 2 servings)
- **2 medium samosas** (store-bought or homemade)
- **1 cup canned chickpeas** (drained and rinsed)
- **½ teaspoon cumin seeds**
- **¼ teaspoon asafoetida (hing)**
- **1 teaspoon ginger paste**
- **½ teaspoon ground coriander powder**
- **¼ teaspoon garam masala**
- **½ teas